In [6]:
# Importing necessary libraries
import os
from PIL import Image
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
import torch
import subprocess
import glob

In [7]:
# Install ultralytics
subprocess.run(["pip", "install", "ultralytics"])
from ultralytics import YOLO

In [8]:
!pip install wandb
import wandb
wandb.login(key='6d205f70d4d770210b103ca36cf1e5cbe0d2f736')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:

# Paths in Colab
image_dir = "/content/drive/MyDrive/Human_Faces_object_Detection/Data/images"
csv_path = "/content/drive/MyDrive/Human_Faces_object_Detection/Data/faces.csv"
output_dir = "/content/output"
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)


In [10]:
import pandas as pd

# Read the CSV file
faces_df = pd.read_csv(csv_path)

# Display the first few rows of the DataFrame
print(faces_df.head())

     image_name  width  height   x0   y0    x1   y1
0  00001722.jpg   1333    2000  490  320   687  664
1  00001044.jpg   2000    1333  791  119  1200  436
2  00001050.jpg    667    1000  304  155   407  331
3  00001736.jpg    626     417  147   14   519  303
4  00003121.jpg    626     418  462   60   599  166


In [11]:
# Plot distributions using Plotly
def plot_distribution(column, title, xlabel):
    fig = px.histogram(faces_df, x=column, nbins=50, title=title,
                       labels={column: xlabel}, template='plotly_dark')
    fig.update_layout(title_font=dict(size=24, family='Arial', color='white'))
    fig.show()

plot_distribution('width', 'Distribution of Image Widths', 'Image Width')
plot_distribution('height', 'Distribution of Image Heights', 'Image Height')
plot_distribution('x0', 'Distribution of X0 Coordinates', 'X0 Coordinate')
plot_distribution('y0', 'Distribution of Y0 Coordinates', 'Y0 Coordinate')
plot_distribution('x1', 'Distribution of X1 Coordinates', 'X1 Coordinate')
plot_distribution('y1', 'Distribution of Y1 Coordinates', 'Y1 Coordinate')


In [12]:
# Function to convert bounding boxes to YOLO format
def convert_to_yolo(row):
    dw = 1. / row['width']
    dh = 1. / row['height']
    x = (row['x0'] + row['x1']) / 2.0
    y = (row['y0'] + row['y1']) / 2.0
    w = row['x1'] - row['x0']
    h = row['y1'] - row['y0']
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return x, y, w, h

In [13]:
# Apply the function to each row and expand the result into new columns
faces_df[['x', 'y', 'w', 'h']] = faces_df.apply(convert_to_yolo, axis=1, result_type='expand')


In [14]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(faces_df, test_size=0.2, random_state=42)


In [15]:
# Function to create YOLO annotation files
def create_annotations(df, annotations_dir):
    os.makedirs(annotations_dir, exist_ok=True)
    for idx, row in df.iterrows():
        annotation_path = os.path.join(annotations_dir, row['image_name'].replace('.jpg', '.txt'))
        with open(annotation_path, 'w') as f:
            f.write(f"0 {row['x']} {row['y']} {row['w']} {row['h']}\n")

In [16]:

# Directory to save the YOLO annotation files
train_annotations_dir = os.path.join(output_dir, 'train', 'labels')
val_annotations_dir = os.path.join(output_dir, 'val', 'labels')


In [17]:

# Create YOLO annotation files for train and validation sets
create_annotations(train_df, train_annotations_dir)
create_annotations(val_df, val_annotations_dir)

In [18]:
# Function to preprocess and save images
def preprocess_images(image_list, source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for img_name in image_list:
        img_path = os.path.join(source_dir, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            target_path = os.path.join(target_dir, img_name)
            cv2.imwrite(target_path, img)

In [19]:
# Define directories for training and validation images
train_images_dir = os.path.join(output_dir, 'train', 'images')
val_images_dir = os.path.join(output_dir, 'val', 'images')


In [20]:
# Preprocess images for train and validation sets
preprocess_images(train_df['image_name'].tolist(), image_dir, train_images_dir)
preprocess_images(val_df['image_name'].tolist(), image_dir, val_images_dir)


In [21]:
# Verify annotation files
print("Train annotations sample:")
print(os.listdir(train_annotations_dir)[:5])

print("Validation annotations sample:")
print(os.listdir(val_annotations_dir)[:5])

Train annotations sample:
['00002867.txt', '00002626.txt', '00001132.txt', '00002511.txt', '00002134.txt']
Validation annotations sample:
['00000089.txt', '00002524.txt', '00001850.txt', '00003002.txt', '00003039.txt']


In [22]:
# Create a YOLOv8 model with the 'yolov8n.pt' pre-trained weights (not 'yolov8n.yaml')
yolo_model = YOLO('yolov8n.pt')


100%|██████████| 6.25M/6.25M [00:00<00:00, 53.6MB/s]


In [23]:
# Define class names
class_names = ['face']


In [24]:
# Construct the content for the data.yaml file
data_yaml_content = f"""
train: {train_images_dir}
val: {val_images_dir}
nc: {len(class_names)}
names: {class_names}
"""


In [25]:
# Path where data.yaml will be saved
data_yaml_path = os.path.join(output_dir, 'data.yaml')


In [26]:
# Save the data.yaml file
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"data.yaml file saved at: {data_yaml_path}")


data.yaml file saved at: /content/output/data.yaml


In [27]:
# Train the model
yolo_model.train(data=data_yaml_path, epochs=10, imgsz=640)

Ultralytics YOLOv8.2.87 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/output/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 755k/755k [00:00<00:00, 10.6MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Currently logged in as: vinaymca91 (vinaymca91-resulticks). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'


...


10 epochs completed in 4.262 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.87 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [01:51<00:00,  5.88s/it]


                   all        585        585      0.722      0.631      0.779      0.528
Speed: 2.3ms preprocess, 171.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/train


lr/pg0,▃▆█▇▆▅▄▃▂▁
lr/pg1,▃▆█▇▆▅▄▃▂▁
lr/pg2,▃▆█▇▆▅▄▃▂▁
metrics/mAP50(B),▁▃▃▆▃▆▇▇██
metrics/mAP50-95(B),▁▃▃▅▂▆▆▇██
metrics/precision(B),▁▃▂▄▂▅▅▇██
metrics/recall(B),▁▄▅▇▆▇█▆▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▆▅▄▄▃▂▂▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e3d74072650>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [29]:
# Function to draw rounded rectangle
def draw_rounded_rectangle(img, start_point, end_point, color, thickness, radius):
    x1, y1 = start_point
    x2, y2 = end_point
    img = cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y2), color, thickness)
    img = cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, thickness)
    img = cv2.circle(img, (x1 + radius, y1 + radius), radius, color, thickness)
    img = cv2.circle(img, (x2 - radius, y1 + radius), radius, color, thickness)
    img = cv2.circle(img, (x1 + radius, y2 - radius), radius, color, thickness)
    img = cv2.circle(img, (x2 - radius, y2 - radius), radius, color, thickness)
    return img


In [37]:
# Display 10 sample images with detection results
def display_samples_with_detections(model, images_dir, samples=10):
    sample_images = os.listdir(images_dir)[:samples]
    for img_name in sample_images:
        img_path = os.path.join(images_dir, img_name)
        results = model(img_path)[0]  # Get the first (and only) Results object
        img = cv2.imread(img_path)
        for box in results.boxes.xyxy.cpu().numpy():  # x1, y1, x2, y2
            x1, y1, x2, y2 = map(int, box)
            img = draw_rounded_rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2, 10)  # Red color, thickness 2, radius 10
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

display_samples_with_detections(yolo_model, val_images_dir)

print("Process completed. Outputs are stored in:", output_dir)

Output hidden; open in https://colab.research.google.com to view.

In [1]:
# Install Git if it’s not installed
!sudo apt-get install git

# Configure your Git username and email
!git config --global user.name "Vinaymca91"
!git config --global user.email "vinaymca91@gmail.com"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!git clone https://github.com/Vinaymca91/Human_Faces_Object_Detection.git


Cloning into 'Human_Faces_Object_Detection'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 4.67 MiB | 11.59 MiB/s, done.


In [3]:
!ls /content/drive/MyDrive/Human_Faces_Object_Detection

detect	output	wandb  yolov8n.pt


In [4]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [5]:
# Configure your Git username and email
!git config --global user.name "Vinaymca91"
!git config --global user.email "vinaymca91@gmail.com"


In [6]:
%cd /content/drive/MyDrive/Human_Faces_Object_Detection


/content/drive/MyDrive/Human_Faces_Object_Detection


In [7]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Human_Faces_Object_Detection/.git/


In [8]:
!git remote add origin https://github.com/Vinaymca91/Human_Faces_Object_Detection


In [9]:
!git add .


In [10]:
!git commit -m "Add dataset and output files"


Auto packing the repository in background for optimum performance.
See "git help gc" for manual housekeeping.
[master (root-commit) 6bf7eef] Add dataset and output files
 4933 files changed, 13927 insertions(+)
 create mode 100644 detect/train/F1_curve.png
 create mode 100644 detect/train/PR_curve.png
 create mode 100644 detect/train/P_curve.png
 create mode 100644 detect/train/R_curve.png
 create mode 100644 detect/train/args.yaml
 create mode 100644 detect/train/confusion_matrix.png
 create mode 100644 detect/train/confusion_matrix_normalized.png
 create mode 100644 detect/train/events.out.tfevents.1725374571.9fa69f00bde9.404.0
 create mode 100644 detect/train/labels.jpg
 create mode 100644 detect/train/labels_correlogram.jpg
 create mode 100644 detect/train/results.csv
 create mode 100644 detect/train/results.png
 create mode 100644 detect/train/train_batch0.jpg
 create mode 100644 detect/train/train_batch1.jpg
 create mode 100644 detect/train/train_batch2.jpg
 create mode 100644 de

In [12]:
!git push -u origin master


fatal: could not read Username for 'https://github.com': No such device or address
